In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import torch

In [8]:
dataset = "fashion-mnist"

In [9]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [10]:
model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=5,
    )

Offnet(
  (sequential): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): OffBlockGroup(
      (sequential): Sequential(
        (0): OffBlock(
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): FakeConv2d(
            (deform): Deform2d(
              (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv): Conv2d(144, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): FakeConv2d(
            (deform): Deform2d(
              (conv): Conv2d(80, 18, kernel_size=(1, 1), stride=(1, 1))
            )
            (conv): Conv2d(720, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (conv_transform): Conv2d(16, 80, kernel_s

In [11]:
net = skorch.net.NeuralNetClassifier(
    module=model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=5,
    ),
    batch_size=32,
    max_epochs=30,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/trainable_wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda',
    train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=Offnet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): OffBlockGroup(
        (sequential): Sequential(
          (0): OffBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(144, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (bn2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(80, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(720, 80, kerne

In [12]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [13]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)

Checkpoint! Saving model to models/trainable_wrn1.pt.
  epoch    train_loss       dur
-------  ------------  --------
      1        0.4514  153.2056
Checkpoint! Saving model to models/trainable_wrn3.pt.
      3        0.2099  155.4873
Checkpoint! Saving model to models/trainable_wrn4.pt.
      4        0.1786  155.4280
Checkpoint! Saving model to models/trainable_wrn5.pt.
      5        0.1519  156.6949
Checkpoint! Saving model to models/trainable_wrn6.pt.
      6        0.1301  156.4875
Checkpoint! Saving model to models/trainable_wrn7.pt.
      7        0.1105  157.4510
Checkpoint! Saving model to models/trainable_wrn8.pt.
      8        0.0933  156.8559
Checkpoint! Saving model to models/trainable_wrn9.pt.
      9        0.0823  157.3233
Checkpoint! Saving model to models/trainable_wrn10.pt.
     10        0.0724  158.4973
Checkpoint! Saving model to models/trainable_wrn11.pt.
     11        0.0640  157.7777
Checkpoint! Saving model to models/trainable_wrn12.pt.
     12        0.05

In [ ]:
prediction_test = net.predict(x_test)

In [ ]:
(prediction_test == y_test).mean()